In [1]:
import sqlite3
import pandas as pd
def create_connection(db_file, verbose=False):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        if verbose:
            print(f'Using SQLite version: {sqlite3.version}')
            print(f'Creating Connection to {db_file}...')
        return conn
    except sqlite3.Error as e:
        print(e)


def select_all_query(db_file, query, verbose=False):
    """
    Query all rows in the database table
    :param db_file: database file
    :return: result as list
    """
    conn = create_connection(db_file, verbose)
    cur = conn.cursor()
    if not query.startswith('SELECT'):
        raise ValueError('Query should begin with `SELECT`')
    
    cur.execute(query)
    rows = cur.fetchall()

    if verbose:
        for row in rows:
            print(row)

    return rows

### Assignment - Part 1, Querying a Database
---

This directory contains a file `rpg_db.sqlite3`, a database for a hypothetical
webapp role-playing game. This test data has dozens-to-hundreds of randomly
generated characters across the base classes (Fighter, Mage, Cleric, and Thief)
as well as a few Necromancers. Also generated are Items, Weapons, and
connections from characters to them. Note that, while the name field was
randomized, the numeric and boolean fields were left as defaults.

Use `sqlite3` to load and write queries to explore the data, and answer the
following questions:

- How many total Characters are there?
- How many of each specific subclass?
- How many total Items?
- How many of the Items are weapons? How many are not?
- How many Items does each character have? (Return first 20 rows)
- How many Weapons does each character have? (Return first 20 rows)
- On average, how many Items does each Character have?
- On average, how many Weapons does each character have?

You do not need all the tables - in particular, the `account_*`, `auth_*`,
`django_*`, and `socialaccount_*` tables are for the application and do not have
the data you need. the `charactercreator_*` and `armory_*` tables and where you
should focus your attention. `armory_item` and `charactercreator_character` are
the main tables for Items and Characters respectively - the other tables are
subsets of them by type (i.e. subclasses), connected via a key (`item_id` and
`character_id`).

You can use the DB Browser or other tools to explore the data and work on your
queries if you wish, but to complete the assignment you should write a file
`rpg_queries.py` that imports `sqlite3` and programmatically executes and
reports results for the above queries.

Some of these queries are challenging - that's OK! You can keep working on them
tomorrow as well (we'll visit loading the same data into PostgreSQL). It's also
OK to figure out the results partially with a query and partially with a bit of
logic or math afterwards, though doing things purely with SQL is a good goal.
[Subqueries](https://www.w3resource.com/sql/subqueries/understanding-sql-subqueries.php)
and [aggregation functions](https://www.sqltutorial.org/sql-aggregate-functions/)
may be helpful for putting together more complicated queries.

1. How many total Characters are there?

In [2]:
results = select_all_query('rpg_db.sqlite3', 'SELECT COUNT(*) FROM charactercreator_character')
print('How many total Characters are there? ', results[0][0])

How many total Characters are there?  302


2. How many of each specific subclass?

**mage**

In [4]:
results = select_all_query('rpg_db.sqlite3', 'SELECT COUNT(*) FROM charactercreator_character as cc INNER JOIN charactercreator_mage as mage on cc.character_id = mage.character_ptr_id')
print(results[0][0])

108


**thief**

In [5]:
results = select_all_query('rpg_db.sqlite3', 'SELECT COUNT(*) FROM charactercreator_character as cc INNER JOIN charactercreator_thief as thief on cc.character_id = thief.character_ptr_id')
print('How many of each thief subclass? ', results[0][0])

How many of each thief subclass?  51


**cleric**

In [6]:
results = select_all_query('rpg_db.sqlite3', 'SELECT COUNT(*) FROM charactercreator_character as cc INNER JOIN charactercreator_cleric as cleric on cc.character_id = cleric.character_ptr_id')
print('How many of each cleric subclass? ', results[0][0])

How many of each cleric subclass?  75


**fighter**

In [7]:
results = select_all_query('rpg_db.sqlite3', 'SELECT COUNT(*) FROM charactercreator_character as cc INNER JOIN charactercreator_fighter as fighter on cc.character_id = fighter.character_ptr_id')
print('How many of each fighter subclass? ', results[0][0])

How many of each fighter subclass?  68


3. How many total Items?

In [8]:
results = select_all_query('rpg_db.sqlite3', 'SELECT COUNT(*) FROM armory_item')
print('How many total items?', results[0][0])

How many total items? 174


4. How many of the Items are weapons? How many are not?

In [9]:
results = select_all_query('rpg_db.sqlite3', 'SELECT COUNT(*) FROM armory_item as item INNER JOIN armory_weapon as weapon ON item.item_id = weapon.item_ptr_id')
print('How many of the items are weapons? ', results[0][0])

How many of the items are weapons?  37


In [10]:
results = select_all_query('rpg_db.sqlite3', 'SELECT (SELECT COUNT(*) FROM armory_item) - (SELECT COUNT(*) FROM armory_item as item INNER JOIN armory_weapon as weapon ON item.item_id = weapon.item_ptr_id)')
print('How many of the items are not weapons? ', results[0][0])

How many of the items are not weapons?  137


5. How many Items does each character have? (Return first 20 rows)

In [12]:
query = '''SELECT character_id as `Character Id`, COUNT(item_id) as `Item Count` 
FROM charactercreator_character_inventory 
GROUP BY character_id LIMIT 20'''
conn = create_connection('rpg_db.sqlite3')
df = pd.read_sql(query, conn)
df

,Character Id,Item Count
0,1,3
1,2,3
2,3,2
3,4,4
4,5,4
5,6,1
6,7,5
7,8,3
8,9,4
9,10,4


6. How many Weapons does each character have? (Return first 20 rows)

In [34]:
query = '''SELECT cci.character_id as `Character Id`, COUNT(aw.item_ptr_id) as `Weapon Count`
FROM charactercreator_character_inventory as cci
INNER JOIN armory_item as ai ON cci.item_id = ai.item_id
INNER JOIN armory_weapon as aw ON ai.item_id = aw.item_ptr_id
GROUP BY cci.character_id
LIMIT 20;'''
conn = create_connection('rpg_db.sqlite3')
df = pd.read_sql(query, conn)
df

,Character Id,Weapon Count
0,5,2
1,7,1
2,11,1
3,20,1
4,22,1
5,23,1
6,26,1
7,27,3
8,29,2
9,30,1


7. On average, how many Items does each Character have?

In [32]:
query = '''SELECT character_id as `Character Id`, COUNT(item_id) as `Item Count` 
FROM charactercreator_character_inventory 
GROUP BY character_id'''
conn = create_connection('rpg_db.sqlite3')
df = pd.read_sql(query, conn)
df.describe()

,Character Id,Item Count
count,302.000000,302.000000
mean,151.500000,2.973510
std,87.324109,1.430317
min,1.000000,1.000000
25%,76.250000,2.000000
50%,151.500000,3.000000
75%,226.750000,4.000000
max,302.000000,5.000000


In [39]:
query = '''SELECT AVG(c)
FROM(
SELECT character_id, COUNT(item_id) as c
FROM charactercreator_character_inventory
GROUP BY character_id
)
'''
conn = create_connection('rpg_db.sqlite3')
df = pd.read_sql(query, conn)
df

,AVG(c)
0,2.97351


8. On average, how many Weapons does each character have?

In [44]:
query = '''SELECT cci.character_id as `Character Id`, COUNT(aw.item_ptr_id) as `Weapon Count`
FROM charactercreator_character_inventory as cci
INNER JOIN armory_item as ai ON cci.item_id = ai.item_id
INNER JOIN armory_weapon as aw ON ai.item_id = aw.item_ptr_id
GROUP BY cci.character_id;'''
conn = create_connection('rpg_db.sqlite3')
df = pd.read_sql(query, conn)
df.describe()

,Character Id,Weapon Count
count,155.000000,155.000000
mean,154.761290,1.309677
std,85.531955,0.553240
min,5.000000,1.000000
25%,80.500000,1.000000
50%,151.000000,1.000000
75%,231.000000,2.000000
max,302.000000,3.000000


In [43]:
query = '''SELECT AVG(wc)
FROM (
SELECT cci.character_id as c, COUNT(aw.item_ptr_id) as wc
FROM charactercreator_character_inventory as cci
INNER JOIN armory_item as ai ON cci.item_id = ai.item_id
INNER JOIN armory_weapon as aw ON ai.item_id = aw.item_ptr_id
GROUP BY cci.character_id
)
'''
conn = create_connection('rpg_db.sqlite3')
df = pd.read_sql(query, conn)
df

,AVG(wc)
0,1.309677


### Assigment - Part 2, Making and populating a Database
---

Load the data (use `pandas`) from the provided file `buddymove_holidayiq.csv`
(the [BuddyMove Data
Set](https://archive.ics.uci.edu/ml/datasets/BuddyMove+Data+Set)) - you should
have 249 rows, 7 columns, and no missing values. The data reflects the number of
place reviews by given users across a variety of categories (sports, parks,
malls, etc.).

Using the standard `sqlite3` module:

- Open a connection to a new (blank) database file `buddymove_holidayiq.sqlite3`
- Use `df.to_sql`
  ([documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html))
  to insert the data into a new table `review` in the SQLite3 database

Then write the following queries (also with `sqlite3`) to test:

- Count how many rows you have - it should be 249!
- How many users who reviewed at least 100 `Nature` in the category also
  reviewed at least 100 in the `Shopping` category?
- (*Stretch*) What are the average number of reviews for each category?

Your code (to reproduce all above steps) should be saved in
`buddymove_holidayiq.py`, and added to the repository along with the generated
SQLite database.

In [54]:
# Create database file if it doesn't exist
with sqlite3.connect('buddymove_holidayiq.sqlite3') as conn:
    
    # 1. Read csv file
    df = pd.read_csv('buddymove_holidayiq.csv')
    
    # 2. DROP TABLE review IF EXISTS
    drop_query = 'DROP TABLE IF EXISTS review'
    conn.cursor().execute(drop_query)
    
    # 3. INSERT TABLE review
    df.to_sql('review', conn, index=False)
    query = 'SELECT * FROM review'
    df = pd.read_sql(query, conn)
df.head()

,User Id,Sports,Religious,Nature,Theatre,Shopping,Picnic
0,User 1,2,77,79,69,68,95
1,User 2,2,62,76,76,69,68
2,User 3,2,50,97,87,50,75
3,User 4,2,68,77,95,76,61
4,User 5,2,98,54,59,95,86


- Count how many rows you have - it should be 249!

In [56]:
query = '''SELECT COUNT(*) FROM review;'''
select_all_query('buddymove_holidayiq.sqlite3', query)[0][0]

249

- How many users who reviewed at least 100 Nature in the category also reviewed at least 100 in the Shopping category?

In [60]:
query = '''SELECT *
FROM review
WHERE `Nature` >= 100 AND `Shopping` >= 100
'''
conn = create_connection('buddymove_holidayiq.sqlite3')
df = pd.read_sql(query, conn)
df.head()

,User Id,Sports,Religious,Nature,Theatre,Shopping,Picnic
0,User 53,4,79,103,68,103,85
1,User 55,8,94,140,89,118,92
2,User 62,8,94,128,89,128,99
3,User 64,6,84,108,138,103,79
4,User 66,6,89,108,128,116,81


- (Stretch) What are the average number of reviews for each category?

In [64]:
query = '''SELECT *
FROM review
'''
conn = create_connection('buddymove_holidayiq.sqlite3')
df = pd.read_sql(query, conn)
df.describe()

,Sports,Religious,Nature,Theatre,Shopping,Picnic
count,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000
mean,11.987952,109.779116,124.518072,116.377510,112.638554,120.401606
std,6.616501,32.454115,45.639372,32.132696,41.562888,32.633339
min,2.000000,50.000000,52.000000,59.000000,50.000000,61.000000
25%,6.000000,84.000000,89.000000,93.000000,79.000000,92.000000
50%,12.000000,104.000000,119.000000,113.000000,104.000000,119.000000
75%,18.000000,132.000000,153.000000,138.000000,138.000000,143.000000
max,25.000000,203.000000,318.000000,213.000000,233.000000,218.000000


In [62]:
query = '''SELECT AVG(Sports), AVG(Religious), AVG(Nature), AVG(Theatre), AVG(Shopping), AVG(Picnic)
FROM review
'''
conn = create_connection('buddymove_holidayiq.sqlite3')
df = pd.read_sql(query, conn)
df

,AVG(Sports),AVG(Religious),AVG(Nature),AVG(Theatre),AVG(Shopping),AVG(Picnic)
0,11.987952,109.779116,124.518072,116.37751,112.638554,120.401606


In [65]:
def get_sql_tables(db_con):
    c = db_con.cursor()
    c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    return c.fetchall()
conn = create_connection('buddymove_holidayiq.sqlite3')
get_sql_tables(conn)

[('review',)]

### Resources and Stretch Goals
---

For a more complicated example SQLite database with a number of tables to play
with, check out this [SQLite Sample
Database](https://www.sqlitetutorial.net/sqlite-sample-database/).

The RPG data also exists in a [JSON
file](https://github.com/LambdaSchool/Django-RPG/blob/master/testdata.json) -
try loading it with the standard [json
module](https://docs.python.org/3.5/library/json.html), and reproducing the
above queries with direct manipulation of the Python dictionaries. Also, try to
load it into a `pandas` dataframe and reproduce the above queries with
appropriate dataframe function calls.

This database is part of a Django (Python webapp framework) application, the
[Django-RPG](https://github.com/LambdaSchool/Django-RPG/tree/master) - check it
out, and (though this is very much a stretch goal) you can [get started with
Django](https://www.djangoproject.com/start/) and see if you can run it
(definitely use `pipenv`!). If you are able to, then you can use the the [Django
ORM](https://docs.djangoproject.com/en/2.1/topics/db/) (object-relational
mapping, a way to interact with SQL through programming language objects), and
[query](https://docs.djangoproject.com/en/2.1/topics/db/queries/) the data.
You'll find that the questions we answered with pure SQL are remarkably simple
to answer using the ORM.

If you need one more stretch goal - the RPG data was generated using
[django-autofixture](https://github.com/volrath/django-autofixture), a tool that
facilitates tests by randomly generating data. Check it out, and if you got
Django working, see if you can generate more data.